In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
import h5py
import pandas as pd

In [ ]:
import labrotation.two_photon_session as tps # TODO: in this cell, kernel dies on other PC
import os

In [ ]:
FOLDER = os.path.normpath("D:\PhD\Data\T386_MatlabTest")

ND2_GREEN_FNAME = "T386_20211202_green.nd2"
ND2_GREEN_LFP = "21d02000.abf"
ND2_GREEN_LV = "T386.021221.1105.txt"
ND2_GREEN_LVTIME = "T386.021221.1105time.txt"
ND2_GREEN_NIK = "T386.021221.1105nik.txt"

ND2_DUAL_FNAME = "T386_20211202_green_red.nd2"
ND2_DUAL_LFP = "21d02001.abf"
ND2_DUAL_LV = "T386.021221.1106.txt"
ND2_DUAL_LVTIME = "T386.021221.1106time.txt"
ND2_DUAL_NIK = "T386.021221.1106_nik.txt"


In [ ]:
MATLAB_2P_PATH = os.path.normpath("D:\\PhD\\matlab-2p\\matlab-2p\\")

# only green channel
nd2_green_fpath = os.path.join(FOLDER, ND2_GREEN_FNAME)
nd2_green_lfp_fpath = os.path.join(FOLDER, ND2_GREEN_LFP)
nd2_green_lv_fpath = os.path.join(FOLDER, ND2_GREEN_LV)
nd2_green_lvtime_fpath = os.path.join(FOLDER, ND2_GREEN_LVTIME)
nd2_green_nik_fpath = os.path.join(FOLDER, ND2_GREEN_NIK)

# export json files (for testing reopening)
nd2_green_full_json = os.path.join(FOLDER, "GREEN_full.json")
nd2_green_full_hd5f = os.path.join(FOLDER, "GREEN_full.h5")
nd2_green_nolfp_json = os.path.join(FOLDER, "GREEN_nolfp.json")
nd2_green_nolfp_hd5f = os.path.join(FOLDER, "GREEN_nolfp.h5")

# green and red channels
nd2_dual_fpath = os.path.join(FOLDER, ND2_DUAL_FNAME)
nd2_dual_lfp_fpath = os.path.join(FOLDER, ND2_DUAL_LFP)
nd2_dual_lv_fpath = os.path.join(FOLDER, ND2_DUAL_LV)
nd2_dual_lvtime_fpath = os.path.join(FOLDER, ND2_DUAL_LVTIME)
nd2_dual_nik_fpath = os.path.join(FOLDER, ND2_DUAL_NIK)

nd2_dual_full_json = os.path.join(FOLDER, "DUAL_full.json")
nd2_dual_full_hd5f = os.path.join(FOLDER, "DUAL_full.h5")
nd2_dual_nolfp_json = os.path.join(FOLDER, "DUAL_nolfp.json")
nd2_dual_nolfp_hd5f = os.path.join(FOLDER, "DUAL_nolfp.h5")

In [ ]:
# TODO: make method to see movement traces matching quality
session = tps.TwoPhotonSession.init_and_process(nd2_path=nd2_green_fpath,
                               nd2_timestamps_path=nd2_green_nik_fpath,
                               labview_path=nd2_green_lv_fpath,
                               labview_timestamps_path=nd2_green_lvtime_fpath,
                               lfp_path=nd2_green_lfp_fpath,
                               matlab_2p_folder=MATLAB_2P_PATH)

In [ ]:
session.export_hdf5(fpath=nd2_green_full_hd5f)

# Open from hdf5

In [ ]:
session2 = tps.TwoPhotonSession.from_hdf5(nd2_green_full_hd5f)

In [ ]:
session2.__dict__.keys() == session.__dict__.keys()

In [ ]:
for key, value in session2.__dict__.items():
    print(f"{key}: {type(value)}")

## print all attributes

In [ ]:
for key, value in session.__dict__.items():
    if isinstance(value, pd.DataFrame):
        print(key)
        for col in value.columns:
            try:
                print(f"\t{col}: {(value[col] == getattr(session2, key)[col]).all()}")
            except ValueError:
                print(f"\t{col}: {(value[col].to_numpy() == getattr(session2, key)[col].to_numpy()).all()}")
    elif isinstance(value, dict):
        for k, v in value.items():
            trues = (v == getattr(session2,key)[k])
            try:  # all() exists only for iterables.
                print(f"{k}:\t{trues.all()}")
            except AttributeError: # for single elements
                print(f"{k}:\t{trues}")
    elif isinstance(value, pd.Series):
        print("Skipping series")
    else:
        print(f"{key}: {value == getattr(session2, key)}")

## print only attributes that do not match

In [ ]:
for key, value in session.__dict__.items():
    if isinstance(value, pd.DataFrame):
        for col in value.columns:
            try:
                if not (value[col] == getattr(session2, key)[col]).all():
                    print(f"{key}:\t{col}:\tFalse")
            except ValueError:
                if not (value[col].to_numpy() == getattr(session2, key)[col].to_numpy()).all():
                    print(f"{key}:\t{col}:\tFalse")
    elif isinstance(value, dict):
        for k, v in value.items():
            trues = (v == getattr(session2,key)[k])
            try:  # all() exists only for iterables.
                if not trues.all():
                    print(f"{key}:\t{k}:\tFalse")
            except AttributeError: # for single elements
                if not trues:
                    print(f"{key}:\t{k}:\tFalse")
    elif isinstance(value, pd.Series):
        if not (value == getattr(session2, key)).all():
            print(f"{key}:\tFalse")
    else:
        if not value == getattr(session2, key):
            print(f"{key}:\tFalse")

In [ ]:
session.nikon_meta["NIDAQ Time [s]"] == session2.nikon_meta["NIDAQ Time [s]"]

In [ ]:
session2.nikon_meta["NIDAQ Time [s]"]

In [ ]:
for col in session.nikon_meta.columns:
    print(sum(session.nikon_meta[col] == session2.nikon_meta[col]) == len(session.nikon_meta))

In [ ]:
len(session.nikon_meta["Unnamed: 5"].dropna())

In [ ]:
for col in session.nikon_meta.columns:
    if len(session.nikon_meta[col].dropna()) < len(session.nikon_meta[col]):
        print(col)

In [ ]:
(session.belt_dict["round"] == session2.belt_dict["round"]).all()

In [ ]:
for key, val in session.belt_dict.items():
    print(f"{key}:\t{(val == session2.belt_dict[key]).all()}")

In [ ]:
session.lfp_df_cut

In [ ]:
session2.lfp_df_cut["t_mov_raw"].to_numpy()

In [ ]:
session.lfp_df_cut["t_mov_raw"].to_numpy()

In [ ]:
(session.lfp_df_cut["t_mov_raw"].to_numpy() == session2.lfp_df_cut["t_mov_raw"].to_numpy()).all()

In [ ]:
(session.nikon_daq_time == session2.nikon_daq_time).all()